In [10]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from sklearn.preprocessing import StandardScaler
from random import choice
import imageio
import os
import cv2
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
plt.style.use('presentation.mplstyle')
import matplotlib as mpl
cmap1 = mpl.cm.get_cmap('spring')
cmap2 = mpl.cm.get_cmap('winter')
def S(t,lamb,C,t0):
    return(C*(1/(1+np.exp(lamb*(t0-t)))))

In [6]:
table[f"ring_density_incr-100_index-{1}"]

Series([], Name: ring_density_incr-100_index-1, dtype: object)

In [11]:
table = time_plate_info.loc[time_plate_info["Plate"]==90].copy()
table

,t,num_trunks,area,area_sep_comp,num_tips,num_nodes,area_study,num_tips_study,num_nodes_study,tot_length_study,...,ring_active_tips_density_incr-200_index-10,ring_active_tips_density_incr-200_index-11,ring_active_tips_density_incr-200_index-12,ring_active_tips_density_incr-200_index-13,ring_active_tips_density_incr-200_index-14,ring_active_tips_density_incr-200_index-15,ring_active_tips_density_incr-200_index-16,ring_active_tips_density_incr-200_index-17,ring_active_tips_density_incr-200_index-18,ring_active_tips_density_incr-200_index-19
1425,0,4,920.520089,32.328281,52,462,831.125114,27,307,51847.917226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426,1,4,952.535331,76.792798,63,519,845.375927,36,316,64221.735379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1427,2,4,972.182055,93.991286,72,648,857.238664,44,416,80280.421604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1428,3,4,991.049024,112.778349,75,764,881.989247,50,501,95028.226997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1429,4,4,1013.736606,133.056938,95,901,904.404381,62,620,111614.224464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1430,5,4,990.765735,152.210811,116,1033,881.82875,86,757,133637.535605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1431,6,4,1094.492337,175.825417,123,1123,941.595155,86,811,149738.648585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1432,7,4,1020.349984,200.421858,135,1194,884.383591,101,869,171755.601403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1433,8,4,1093.365965,230.826025,158,1412,958.465385,122,1051,205044.62226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1434,9,4,1159.587138,258.91337,189,1641,1025.127213,143,1266,237598.02632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
plt.close('all')
plate = 90
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
table= table.set_index('t')

ts = []
ys = []
ys2 = []
max_area = 50

for index in range(1,maxi):
    column = f"ring_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[column]>=500]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        area = np.sqrt(table["area_sep_comp"][0]+100*index)
        selection_fit = table.loc[table[f'time_since_begin_{index}'].between(-30,30)]
        popt0, pcov = curve_fit(S, selection_fit[f'time_since_begin_{index}'],selection_fit[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
        lamb,C,t0 = list(popt0)
        table[f'time_since_begin_{index}']=table['time_since_begin']-start -t0
        
        ax.scatter(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
        ax.plot(table[f'time_since_begin_{index}'],S(table[f'time_since_begin_{index}'],lamb,C,0),color=cmap2(area/max_area),alpha = 0.2)
        
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
selection_fit[column]

t
0             0.0
1             0.0
2       17.995891
3             0.0
4             0.0
5             0.0
6             0.0
7       49.173244
8             0.0
9             0.0
10            0.0
11      48.940632
12     165.788666
13     571.446185
14    1085.223094
15    1329.530792
16    1564.402751
17    1730.817853
18     1948.55357
19    1853.332865
20    1996.277417
21    1843.398798
22    2053.926607
23    1994.488821
24    2090.857829
25    2443.062465
26    2633.990422
27    2190.112462
28    1809.538269
29    1792.895132
30    1832.337413
31    2306.395518
32    2571.728797
33     2764.90919
34    2770.890779
35    2789.886397
36    2740.836018
37    2738.933664
38    2848.380852
39    2834.068351
Name: ring_density_incr-100_index-5, dtype: object

In [39]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
for plate in [94,152,792,799,800,127,99,90]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []


    for index in range(maxi):
        column = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column]>=np.max(table[column])//2]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            selection_fit = table.loc[table[f'time_since_begin_{index}'].between(-30,30)]
            popt0, pcov = curve_fit(S, selection_fit[f'time_since_begin_{index}'],selection_fit[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
            lamb,C,t0 = list(popt0)
            table[f'time_since_begin_{index}']=table['time_since_begin']-start -t0
        
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
    ax.plot(meancurve.index,meancurve,label=plate)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,label=plate,linestyle="dashed")
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.legend()
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
